# Q# #

IQ# krävs för att köra denna Notebook.

Q# är utvecklat av Microsoft, designat för kvantprogrammering och är menat att interagera med ett annat, klassiskt programmeringsspråk.

I **Q#** skriver man så kallade **operationer**, eller subrutiner, som gör kvantberäkningar. Dessa kan sedan köras från
ett klassiskt programmeringsspråk. Vi börjar med det enklaste av exempel.

In [1]:
operation HelloWorld() : Unit {
    Message("Hello World!");
}

HelloWorld

För att sedan köra detta måste vi simulera kvantbitar, då vi inte har en kvantdator.

In [2]:
%simulate HelloWorld

Hello World!


()

# Kvantbitar i Q# #

Än så länge är detta ingenting annorlunda, så låt oss kliva in i kvantvärlden genom att skaffa oss en kvantbit.

In [3]:
operation QubitFlip() : Unit {
    
    // Get one qubit
    using (qubit = Qubit()) {
        
        // Qubits are initialized as |0>
        let measure_first = M(qubit);
        Message($"Initial measure: {measure_first}");
        
        // Flip it to |1>
        X(qubit);
        
        // Measure to verify that it is flipped.
        let measure_second = M(qubit);
        Message($"After flip: {measure_second}");
        
        // Reset to |0>
        X(qubit);
    }
}

QubitFlip

In [4]:
%simulate QubitFlip

Initial measure: Zero
After flip: One


()

# Superposition

Som vi vet kan kvantbitar vara i superposition mellan **0** och __1__, så låt oss se hur detta fungerar i Q#.

In [5]:
operation MeasureSuperposition(qubit : Qubit) : Result {
    // Assuming the qubit is initialized as |0>
    
    // Change the qubit to a superposition state with a Hadamard gate
    H(qubit);
    
    // Measure the qubit. This collapses the superposition and
    // should measure |1> 50% of the time.
    let measure = M(qubit);
    
    return measure;
}

MeasureSuperposition

In [6]:
// Need to bring in the Convert namespace to access ResultArrayAsInt
open Microsoft.Quantum.Convert;

operation TestMeasurement() : Unit {

    // Measure an even superposition 10,000 times.
    let iterations = 10000;
    mutable sum = 0;

    using (qubit = Qubit()) {
        // Qubits are initialized to |0>
        
        for (i in 0..iterations - 1) {
        
            // Create a superposition and measure it.
            let result = MeasureSuperposition(qubit);
            
            set sum = sum + ResultArrayAsInt([result]);
            
            // Reset the qubit to |0>
            if (M(qubit) == One) {
                X(qubit);
            }
        }
    }
    
    Message($"Ones measured: {sum}");
    Message($"Zeros measured: {iterations - sum}");
}

TestMeasurement

In [7]:
%simulate TestMeasurement

Ones measured: 4947
Zeros measured: 5053


()

# Sammanflätning

En av de stora fördelarna med kvantprogrammering är att du kan sammanfläta två kvantpartiklar. Deras tillstånd är
då beroende av den andras. Man kan genomföra detta med en **Hadamard-grind** och en __CNOT-grind__.

In [8]:
operation Entangle(qubit1: Qubit, qubit2: Qubit) : Unit {
    
    // Put the first qubit in |+> with a Hadamard gate
    H(qubit1);
    
    // Make a controlled NOT with the second qubit as target,
    // and the first qubit as control
    CNOT(qubit1, qubit2);
    
    // These qubits should now be entangled.
}

operation Reset(qubit: Qubit) : Unit {
    // Resets the qubit to the |0> state.
    if (M(qubit) == One) {
        X(qubit);
    }
}

Entangle, Reset

Vi kan nu testa att dessa faktiskt är sammanflätade. Om de är det så borde fördelningen mellan 1 och 0 om vi mäter första kvantbiten fortfarande vara runt 50%, men varje gång vi mäter 1 på första kvantbiten så borde vi även mäta 1 från andra kvantbiten (samma sak gäller för 0).

In [9]:
open Microsoft.Quantum.Convert;

operation TestEntanglement() : Unit {

    // Measure an even superposition 10,000 times.
    let iterations = 10000;
    
    // The sum of the measured value of the first qubit.
    mutable sum = 0;
    
    // The times the first qubit measure equal the second qubit measure.
    mutable times_equal = 0;

    using ((qubit1, qubit2) = (Qubit(), Qubit())) {
        // Qubits are initialized to |0>
        
        for (i in 0..iterations - 1) {
        
            // Entangle the qubits
            Entangle(qubit1, qubit2);
            
            // Measure the first qubit and increment the sum
            let measure1 = M(qubit1);
            set sum = sum + ResultArrayAsInt([measure1]);
            
            // Measure the second qubit
            let measure2 = M(qubit2);
            if (measure1 == measure2) {
                set times_equal = times_equal + 1;
            }
            
            // Reset the qubits to |0>
            Reset(qubit1); Reset(qubit2);
        }
    }
    
    Message($"Ones measured: {sum}");
    Message($"Zeros measured: {iterations - sum}");
    Message($"Times measured equal: {times_equal}");
}

TestEntanglement

In [10]:
%simulate TestEntanglement

Ones measured: 4952
Zeros measured: 5048
Times measured equal: 10000


()

# Ett mer konkret exempel: Superdense Coding

Vi kommer nu gå igenom stegen för att implementera superdense coding (supertät kodning) med kvantbitar. Superdense Coding innebär att två personer, Alice och Bob, kan kommunicera två bitar information genom att enbart skicka en kvantbit till den andra, med antagandet att de båda hade var sin partikel av ett sammanflätat par.

In [14]:
open Microsoft.Quantum.Math;

operation Superdense() : Unit {

    using ( (AliceQ, BobQ) = (Qubit(), Qubit()) ) {
        // Sammanfläta kvantbitarna.
        Entangle(AliceQ, BobQ);
        
        // Vid det här laget kan Alice och Bob gå skilda vägar, och
        // tar med sig sin egna kvantbit.
        
        // Senare vill Alice skicka ett meddelande.
        let message = [RandomInt(2), RandomInt(2)];
        Message($"Message: {message}");

        // Alice kodar meddelandet i sin kvantbit.
        Encode(AliceQ, message);

        // Hon skickar sedan sin kvantbit till Bob.
        
        // Först avkodar Bob meddelandet.
        Decode(AliceQ, BobQ);
        
        // Bob kan då extrahera meddelandet ur sin och Alices kvantbit.
        ExtractMessage(AliceQ, BobQ);

        // Återställ alla kvantbitar till |0>.
        Reset(AliceQ); Reset(BobQ); 
    } 
}

Superdense

## Kodning av meddelandet

Alice kan koda meddelandet på följande sätt:

 - Om hon vill skicka |00> så gör hon ingenting med sin kvantbit.
 - Om hon vill skicka |01> så utför hon X på sin kvantbit.
 - Om hon vill skicka |10> så utför hon Z på sin kvantbit.
 - Om hon vill skicka |11> så utför hon både Z och X på sin kvantbit.

In [11]:
operation Encode(AliceQ : Qubit, message : Int[]) : Unit {
    // Koda meddelandet i Alice kvantbit.
    if (message[0] == 0 and message[1] == 0) {
        // Gör ingenting
    } elif (message[0] == 0 and message[1] == 1) {
        X(AliceQ);
    } elif (message[0] == 1 and message[1] == 0) {
        Z(AliceQ);
    } elif (message[0] == 1 and message[1] == 1) {
        Z(AliceQ);
        X(AliceQ);
    }
}

Encode

In [12]:
operation Decode(AliceQ : Qubit, BobQ : Qubit) : Unit {
    // Avkoda meddelandet med båda kvantbitarna.
    CNOT(AliceQ, BobQ);
    H(AliceQ);
}

Decode

In [13]:
operation ExtractMessage(AliceQ : Qubit, BobQ : Qubit) : Unit {
    // Extrahera meddelandet genom att mäta kvantbitarna.
    let alice = M(AliceQ);
    let bob = M(BobQ);
    Message($"Received: {alice}, {bob}");
}

ExtractMessage

In [27]:
%simulate Superdense

Message: [1,0]
Received: One, Zero


()